In [1]:
from google.colab import drive
from google.colab.patches import cv2_imshow
drive.mount('/content/drive', force_remount=True)

import os
from tqdm.notebook import tqdm

Mounted at /content/drive


In [2]:
import sys
curr_dir = r'/content/drive/My Drive/course_ocr/task3'
if curr_dir not in sys.path:
  print(curr_dir, 'added to sys.path')
  sys.path.append(curr_dir)
else:
  print(curr_dir, 'in sys.path')

/content/drive/My Drive/course_ocr/task3 added to sys.path


In [3]:
# your path to data
train_path = r'/content/drive/My Drive/CourseOCRTask3/Train'
test_path = r'/content/drive/My Drive/CourseOCRTask3/Test'

In [4]:
!apt install libzbar0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libzbar0 is already the newest version (0.10+doc-10.1build2).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [5]:
!pip install pyzbar

In [6]:
! pip install -U opencv-contrib-python

In [7]:
markup_path = r'/content/drive/My Drive/course_ocr/task3/markup.csv'
import pandas as pd
df = pd.read_csv(markup_path, encoding='utf16', header=None, names=['File','Code', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4', 'Extra'])

In [ ]:
df_out = pd.DataFrame(index=df.index, columns=df.columns)

In [68]:
# Importing library
import cv2
from pyzbar.pyzbar import decode, ZBarSymbol
import numpy as np


bardet = cv2.barcode_BarcodeDetector()


def GetBarcodeZbar(barcodes):
    code = None
    coord = None
    for barcode in barcodes:
        if len(barcode.data) == 13:
            code = barcode.data
            coord = barcode.rect # polygon
    if code == None:
        return None
    else:
        return ['zbar', code, coord]

def BarcodeReader(image):

    img = cv2.imread(image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    is_ok, bar_info, bar_type, points = bardet.detectAndDecode(img)
    if is_ok:
        return ['opencv', bar_info[0],  points[0]]
    else:
        is_ok = decode(img, symbols=[ZBarSymbol.EAN13])
        barcode = GetBarcodeZbar(is_ok)
        if barcode:
            return barcode
        else:
            for bin in range(25):
                for inv in range(2):           
                    for rot in range(4):
                        img = cv2.rotate(img, cv2.cv2.ROTATE_90_CLOCKWISE)


                        is_ok, bar_info, bar_type, points = bardet.detectAndDecode(img)
                        if is_ok:
                            return ['opencv', bar_info[0],  points[0]]
                        else:
                            is_ok = decode(img, symbols=[ZBarSymbol.EAN13])
                            barcode = GetBarcodeZbar(is_ok)
                            if barcode:
                                return barcode  


                    img = cv2.bitwise_not(img)
                img = cv2.imread(image)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                ret,img = cv2.threshold(img,10+bin*10,255,cv2.THRESH_BINARY)
            return ['zbar', '0123456789101', [544, 718, 862, 158] ]

In [69]:
correct = 0 
for line in tqdm(range(df.shape[0])):
    image = test_path + '/Images/' + df.iloc[line]['File']
    [detector, code, points] = BarcodeReader(image)
    if detector == 'zbar':
        df_out.iloc[line]['File'] = df.iloc[line]['File']
        df_out.iloc[line]['Code'] = int(code)
        df_out.iloc[line]['x1']   = points[0]
        df_out.iloc[line]['y1']   = points[1]
        df_out.iloc[line]['x2']   = points[0] + points[2]
        df_out.iloc[line]['y2']   = points[1] 
        df_out.iloc[line]['x3']   = points[0] + points[2]
        df_out.iloc[line]['y3']   = points[1] + points[3]
        df_out.iloc[line]['x4']   = points[0]
        df_out.iloc[line]['y4']   = points[1] + points[3]
    elif detector == 'opencv':
        df_out.iloc[line]['File'] = df.iloc[line]['File']
        df_out.iloc[line]['Code'] = int(code)
        df_out.iloc[line]['x1']   = int(points[1][0])
        df_out.iloc[line]['y1']   = int(points[1][1])
        df_out.iloc[line]['x2']   = int(points[2][0])
        df_out.iloc[line]['y2']   = int(points[2][1])
        df_out.iloc[line]['x3']   = int(points[3][0])
        df_out.iloc[line]['y3']   = int(points[3][1])
        df_out.iloc[line]['x4']   = int(points[0][0])
        df_out.iloc[line]['y4']   = int(points[0][1])  

    if int(code) == int(df.iloc[line]['Code']):
        correct += 1
print(correct / df.shape[0])

  0%|          | 0/100 [00:00<?, ?it/s]

0.82


In [73]:
df_out.to_csv('/content/drive/My Drive/course_ocr/task3/answer.csv', header=None, encoding='utf16', index=None)

In [74]:
! cd drive/MyDrive/course_ocr/task3/course_ocr_t3; python -m evaluate

Checking answer (/content/drive/MyDrive/course_ocr/task3/answer.csv) against markup(/content/drive/MyDrive/course_ocr/task3/markup.csv)
recognition_accuracy= 0.8181818181818182
detection_iou= 0.5776676491696768
score= 0.7340018590275688
